# Practial exercises with PyGLDA toolbox
---
Edited by Fan Yang (fany@plan.aau.dk) and all other members from AAU geodesy group. This document is dedicated to the summer school held by Geodesy Group, Aalborg University, Denmark. This can be used as a quick start of the PyGLDA toolbox.

**Notice**: please makre sure that this notebook is opened under the environment of **pyglda**, following these steps: navigate to the '../PyGLDA/demo/..', open a terminal and type
> conda activate pyglda
> 
> pip install jupyterlab
> 
> jupiter lab
---

# Exercise-2
The task is to test the impact of choosing differeng GRACE solutions: the options are GRACE-SH and GRACE-Mascon solution.

---


## 1. Load the default settings

In [1]:
from demo_summer_school import RDA, load_main_configuration_parameters,save_main_configuration_parameters
load_main_configuration_parameters()

{'basin': 'Brahmaputra3subbasins',
 'shp_path': '../data/basin/shp/Brahmaputra/Brahmaputra3subbasins_subbasins.shp',
 'case': 'Exercise_2',
 'setting_dir': '../settings/SummerSchool/Exercise_2',
 'ens': 10,
 'cold_begin_time': '2004-10-01',
 'cold_end_time': '2004-12-31',
 'warm_begin_time': '2005-01-01',
 'warm_end_time': '2005-12-31',
 'resume_begin_time': '2005-02-01',
 'resume_end_time': '2005-12-31',
 'external_data_path': '/media/user/My Book/Fan/SummerSchool/External Data',
 'res_output': '/media/user/My Book/Fan/SummerSchool/External Data/w3ra/res',
 'figure_output': '/media/user/My Book/Fan/SummerSchool/External Data/w3ra/figure'}

## 2. Preview of the setting file that controls the observation 

In [2]:
RDA.setting_dir = '../settings/SummerSchool/Exercise_2'
import json
from pathlib import Path

## GRACE-MASCON
config_mascon= json.load(open(Path(RDA.setting_dir)/'DA_setting_Mascon.json', 'r'))
config_mascon['obs']['GRACE']

{'EWH_grid_dir': '/media/user/My Book/Fan/SummerSchool/External Data/GRACE/SaGEA/signal_Mascon',
 'cov_dir': '/media/user/My Book/Fan/SummerSchool/External Data/GRACE/SaGEA/sample_DDK3',
 'preprocess_res': '/media/user/My Book/Fan/SummerSchool/External Data/GRACE/output/',
 'OL_mean': '/media/user/My Book/Fan/SummerSchool/External Data/GRACE/OLmean',
 'aux_for_time_epochs': '/media/user/My Book/Fan/SummerSchool/External Data/GRACE/SaGEA/signal_Mascon',
 'kind': 'Mascon_monthly'}

In [3]:
## GRACE-SH
config_SH= json.load(open(Path(RDA.setting_dir)/'DA_setting_SH.json', 'r'))
config_SH['obs']['GRACE']

{'EWH_grid_dir': '/media/user/My Book/Fan/SummerSchool/External Data/GRACE/SaGEA/signal_DDK3',
 'cov_dir': '/media/user/My Book/Fan/SummerSchool/External Data/GRACE/SaGEA/sample_DDK3',
 'preprocess_res': '/media/user/My Book/Fan/SummerSchool/External Data/GRACE/output/',
 'OL_mean': '/media/user/My Book/Fan/SummerSchool/External Data/GRACE/OLmean',
 'aux_for_time_epochs': '/media/user/My Book/Fan/SummerSchool/External Data/GRACE/SaGEA/signal_DDK3',
 'kind': 'SH_monthly'}

## 3. Make your choice of GRACE solution

In [4]:
### modify the settings and save to the json###
config = config_mascon
# config = config_SH
with open(Path(RDA.setting_dir)/'DA_setting.json', 'w') as f:
    json.dump(config, f, indent=4)

### start a new case study to avoid confusion ###
RDA.case= 'Exercise_2'
save_main_configuration_parameters()

{'basin': 'Brahmaputra3subbasins',
 'shp_path': '../data/basin/shp/Brahmaputra/Brahmaputra3subbasins_subbasins.shp',
 'case': 'Exercise_2',
 'setting_dir': '../settings/SummerSchool/Exercise_2',
 'ens': 10,
 'cold_begin_time': '2004-10-01',
 'cold_end_time': '2004-12-31',
 'warm_begin_time': '2005-01-01',
 'warm_end_time': '2005-12-31',
 'resume_begin_time': '2005-02-01',
 'resume_end_time': '2005-12-31',
 'external_data_path': '/media/user/My Book/Fan/SummerSchool/External Data',
 'res_output': '/media/user/My Book/Fan/SummerSchool/External Data/w3ra/res',
 'figure_output': '/media/user/My Book/Fan/SummerSchool/External Data/w3ra/figure'}

## 4. Re-generate GRACE data

In [5]:
from demo_summer_school import demo_prepare_GRACE_Mascon, demo_prepare_GRACE
if config['obs']['GRACE']['kind'] == 'Mascon_monthly':
    demo_prepare_GRACE_Mascon(isDiagonal=False)
elif config['obs']['GRACE']['kind'] == 'SH_monthly':
    demo_prepare_GRACE(isDiagonal=False)

Start to pre-process GRACE to obtain basin-wise TWS over places of interest...
2005-01
2005-02
2005-03
2005-04
2005-05
2005-06
2005-07
2005-08
2005-09
2005-10
2005-11
2005-12
Finished: 2025-08-18 15:31:33
Start to pre-process GRACE to obtain grid-wise TWS over places of interest...
2005-01
2005-02
2005-03
2005-04
2005-05
2005-06
2005-07
2005-08
2005-09
2005-10
2005-11
2005-12
Finished: 2025-08-18 15:31:34

Start to pre-process GRACE to obtain COV over places of interest...
2005-01_0.npy
2005-02_0.npy
2005-03_0.npy
2005-04_0.npy
2005-05_0.npy
2005-06_0.npy
2005-07_0.npy
2005-08_0.npy
2005-09_0.npy
2005-10_0.npy
2005-11_0.npy
2005-12_0.npy
Finished: 2025-08-18 15:31:46


## 4. Perform the open loop and data assimilation

In [6]:
RDA.single_run()
!mpiexec -n 11 python -u demo_summer_school.py OL
!mpiexec -n 11 python -u demo_summer_school.py DA


Configuring W3v2 run
Spatial resolution: 0.1
Area: lat- [31.2, 21.8], lon- [87.8, 97.7]
Forcing field: ERA5

Initializing model states: cold
Guess cold states.
Spin up run for 2004-10-01 to 2004-12-31 (92 days)

===== RUN from 2004-10-01 to 2004-12-31 =====

Doing year/month 200410
...............................
Doing year/month 200411
..............................
Doing year/month 200412
...............................
Finished

Configuring W3v2 run
Spatial resolution: 0.1
Area: lat- [31.2, 21.8], lon- [87.8, 97.7]
Forcing field: ERA5
/bin/bash: line 1: mpiexec: command not found
/bin/bash: line 1: mpiexec: command not found


---
### 5. Please go to another notebook for the visualization ==> **Visualization.ipynb**